In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Load training and validation sets
ds_train_ = image_dataset_from_directory(
    '../input/utkface-new',
    validation_split=0.2,
    subset="training",
    seed=1337,
    labels='inferred',
    image_size=[128, 128],
    interpolation='nearest',
    batch_size=64,
    shuffle=True,
)

ds_valid_ = image_dataset_from_directory(
    '../input/utkface-new',
    validation_split=0.2,
    subset="validation",
    seed=1337,
    labels='inferred',
    image_size=[128, 128],
    interpolation='nearest',
    batch_size=64,
    shuffle=False
)

# Data Pipeline
def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE
ds_train = (
    ds_train_
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)
ds_valid = (
    ds_valid_
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)


